# Xác định sốt rét ở người

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Thiết lập

### Gói dữ liệu về bệnh sốt rét

# Định dạng của đoạn này là mã

Nhóm xem tải dataset ở đường link này nhé:
https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria



### Nạp thư viện

In [3]:
# nhập các thư viện như được hiển thị bên dưới
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from glob import glob

import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
# Thiết lập kích thước ảnh
IMAGE_SIZE = [224, 224]

## Xử lý dữ liệu

In [4]:
# Nhập thư viện VGG19 và thêm lớp tiền xử lý vào phía trước của VGG
# Ở đây mình sẽ sử dụng trọng số imagenet
mobilnet = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 1s 0us/step


In [5]:
# Không cho train khi đã có weights
for layer in mobilnet.layers:
    layer.trainable = False

In [6]:
# Dùng đếm số lượng -> từ đường dẫn thư mục
folders = glob('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/*')
# Lớp xử lý của mình ở đây -> có thể thêm nếu thích
x = Flatten()(mobilnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

### Tạo model

In [7]:
# Tạo model
model = Model(inputs=mobilnet.input, outputs=prediction)
# Xem cấu trúc mô hình
# model.summary()
# có nhu cầu thì mở ra coi

####Tạo model từ đầu bằng CNN

In [8]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))
# có nhu cầu thì mở ra coi
# model.summary()

In [9]:
# Cho biết phương pháp tối ưu hóa được sử dụng
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

#Sử dụng Trình tạo Dữ liệu Hình ảnh để nhập hình ảnh từ tập dữ liệu
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

### Cấu hình lại dữ liệu

In [10]:
# Đảm bảo cung cấp kích thước mục tiêu giống như đã khởi tạo cho kích thước hình ảnh
# KHÔNG XÀI TỆP 500MB, KHÔNG XÀI TỆP 500MB, XÀI LÀ TRAIN TỚI SÁNG MỐT
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/Test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/Train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

#KHÔNG XÀI TỆP NÀY TRAIN
# training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/WARNING_Malaria_SotRetFullDataSet500MB/',
#                                                  target_size = (224, 224),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')


Found 134 images belonging to 2 classes.
Found 635 images belonging to 2 classes.


###TRANINGGGGGGGGGGGGGGGGGGGGGG

In [11]:
#Thực thi cho máy học, có thể tủy chỉnh epoch để tăng thời gian và độ chính xác
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  # epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
20/20 [==============================] - 168s 8s/step - loss: 1.1025 - accuracy: 0.5858 - val_loss: 0.7534 - val_accuracy: 0.3881
Epoch 2/10
20/20 [==============================] - 29s 1s/step - loss: 0.5455 - accuracy: 0.7260 - val_loss: 0.8736 - val_accuracy: 0.3806
Epoch 3/10
20/20 [==============================] - 29s 1s/step - loss: 0.5014 - accuracy: 0.7780 - val_loss: 1.0244 - val_accuracy: 0.4104
Epoch 4/10
20/20 [==============================] - 29s 1s/step - loss: 0.4987 - accuracy: 0.7748 - val_loss: 0.8765 - val_accuracy: 0.4403
Epoch 5/10
20/20 [==============================] - 29s 1s/step - loss: 0.4462 - accuracy: 0.8047 - val_loss: 0.8021 - val_accuracy: 0.5299
Epoch 6/10
20/20 [==============================] - 29s 1s/step - loss: 0.4156 - accuracy: 0.8425 - val_loss: 0.7382 - val_accuracy: 0.5672
Epoch 7/10
20/20 [==============================] - 29s 1s/step - loss: 0.3615 - accuracy: 0.8551 - val_loss: 0.8439 - val_accuracy: 0.5373
Epoch 8/10
20/20 [=

###Biểu đồ

In [19]:
# # này của độ phương sai 
# plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
# plt.legend()
# plt.show()
# plt.savefig('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/pickle/result/LossVal_loss')

# # này của độ chính xác
# plt.plot(r.history['accuracy'], label='train acc')
# plt.plot(r.history['val_accuracy'], label='val acc')
# plt.legend()
# plt.show()
# plt.savefig('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/pickle/result/AccVal_acc')

##DỮ LIỆU ~750 HÌNH, ĐỘ CHÍNH XÁC CÓ VẺ CHƯA CAO, SẼ THÊM DATASET SAU. TẠM THƯỜI BỎ QUA PHẦN BIỂU ĐỒ NÀY
plt.style.use("seaborn")
plt.plot(r.history['accuracy'],label="acc")
plt.plot(r.history['val_accuracy'],label="val_acc")
plt.plot(r.history['loss'],label="loss")
plt.plot(r.history['val_loss'],label="val_loss")
plt.legend()
plt.plot()

[]

In [20]:
plt.savefig('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/pickle/result/Va2l_acc')

In [14]:
dim1_ = []
dim2_ = []

# for image_filename in os.listdir('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/WARNING_Malaria_SotRetFullDataSet500MB/Nhiem/'):
#     if 'Thumbs.db' not in image_filename: 
#         img = imread('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/WARNING_Malaria_SotRetFullDataSet500MB/Nhiem/' + image_filename)
#         d1,d2,colors = img.shape
#         dim1_.append(d1)
#         dim2_.append(d2)

for image_filename in os.listdir('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/Train/Parasite/'):
    if 'Thumbs.db' not in image_filename: 
        img = imread('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/Train/Parasite/' + image_filename)
        d1,d2,colors = img.shape
        dim1_.append(d1)
        dim2_.append(d2)

# # sơ đồ Jointplot KDE
# sns.jointplot(dim1_,dim2_,kind="reg")

# # sơ đồ Jointplot Classic
# sns.jointplot(dim1_,dim2_,kind="hist")

# Sơ đồ phối hợp
g = sns.jointplot(dim1_,dim2_,kind="reg")
g.plot_joint(sns.kdeplot,color="r", zorder=0, levels=10)
g.plot_marginals(sns.rugplot, color="y", height=-.15, clip_on=False)

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [15]:
# Lưu file .h5
model.save('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/pickle/Malaria_SotRet.h5')

In [16]:
# này để sau này phát sinh
# y_pred = model.predict(test_set)
# y_pred

#Load file .h5 để xài

In [17]:
model=load_model('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/pickle/Malaria_SotRet.h5')

In [18]:
img=image.load_img('/content/drive/MyDrive/Colab Notebooks/NMT_VLUTE/Datasets/Malaria_SotRet/Train/Parasite/C101P62ThinF_IMG_20150923_170110_cell_14.png',target_size=(224,224))
x=image.img_to_array(img)
x=x/255
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
model.predict(img_data)
a=np.argmax(model.predict(img_data), axis=1)
print(a)
if(a==1):
    print("Không bị gì hết nè")
else:
    print("Bé bị sốt rét rồi bé ơi!")

[0]
Bé bị sốt rét rồi bé ơi!
